In [35]:
import pandas as pd
import os 
import time 
from binance.client import Client
from binance.enums import *

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
api_key = os.getenv("KEY_BINANCE")
secret_key = os.getenv("SECRET_BINANCE")

cliente_binance = Client(api_key, secret_key)

In [40]:
codigo_operado = "SOLBRL"
ativo_operado = "SOL"
periodo_candle = Client.KLINE_INTERVAL_1HOUR
quantidade = 0.015

In [45]:
def pegando_dados(codigo, intervalo):

    candles = cliente_binance.get_klines(symbol = codigo, interval = intervalo, limit = 1000)
    precos = pd.DataFrame(candles)
    precos.columns = ["tempo_abertura", "abertura", "maxima", "minima", "fechamento", "volume", "tempo_fechamento", "moedas_negociadas", "numero_trades",
                    "volume_ativo_base_compra", "volume_ativo_cotação", "-"]
    precos = precos[["fechamento", "tempo_fechamento"]]
    precos["tempo_fechamento"] = pd.to_datetime(precos["tempo_fechamento"], unit = "ms").dt.tz_localize("UTC")
    precos["tempo_fechamento"] = precos["tempo_fechamento"].dt.tz_convert("America/Sao_Paulo")

    return precos

In [47]:
pegando_dados(codigo_operado, periodo_candle)

,fechamento,tempo_fechamento
0,1070.10000000,2024-11-07 00:59:59.999000-03:00
1,1069.50000000,2024-11-07 01:59:59.999000-03:00
2,1068.50000000,2024-11-07 02:59:59.999000-03:00
3,1062.80000000,2024-11-07 03:59:59.999000-03:00
4,1066.40000000,2024-11-07 04:59:59.999000-03:00
...,...,...
995,1331.50000000,2024-12-18 11:59:59.999000-03:00
996,1351.10000000,2024-12-18 12:59:59.999000-03:00
997,1343.20000000,2024-12-18 13:59:59.999000-03:00
998,1348.60000000,2024-12-18 14:59:59.999000-03:00


In [48]:
def estrategia_trade(dados, codigo_ativo, ativo_operado, quantidade, posicao):

    dados["media_rapida"] = dados["fechamento"].rolling(window = 7).mean()
    dados["media_devagar"] = dados["fechamento"].rolling(window = 40).mean()

    ultima_media_rapida = dados["media_rapida"].iloc[-1]
    ultima_media_devagar = dados["media_devagar"].iloc[-1]

    print(f"Última Média Rápida: {ultima_media_rapida} | Última Média Devagar: {ultima_media_devagar}")

    conta = cliente_binance.get_account()

    for ativo in conta["balances"]:

        if ativo["asset"] == ativo_operado:

            quantidade_atual = float(ativo["free"])

    if ultima_media_rapida > ultima_media_devagar:

        if posicao == False:

            #order = cliente_binance.create_order(symbol = codigo_ativo, side = SIDE_BUY, type = ORDER_TYPE_MARKET, quantity = quantidade)
            
            print("COMPROU O ATIVO")

            posicao = True

    elif ultima_media_rapida < ultima_media_devagar:

        if posicao == True:

            #order = cliente_binance.create_order(symbol = codigo_ativo, side = SIDE_SELL, type = ORDER_TYPE_MARKET,quantity = int(quantidade_atual * 1000)/1000)
            
            print("VENDER O ATIVO")

            posicao = False

    return posicao

In [51]:
posicao_atual = True

In [55]:
while True:
    dados_atualizados = pegando_dados(codigo=codigo_operado, intervalo=periodo_candle)
    posicao_atual = estrategia_trade(dados_atualizados, codigo_ativo=codigo_operado, ativo_operado=ativo_operado, quantidade=quantidade, posicao=posicao_atual)
    time.sleep(60 * 20)

Última Média Rápida: 1342.3999999999999 | Última Média Devagar: 1351.735


BinanceAPIException: APIError(code=-1013): Invalid quantity.